In [1]:
%run -i ../code/wavelets_pca_with_zhost_1100_average_precision.py

In [2]:
%run -i ../code/wavelets_pca_with_zhost_1100_roc_auc.py

In [3]:
import pandas as pd
import numpy as np

import pywt as wt

import pymc3 as pm
%matplotlib inline

import matplotlib.pyplot as plt

In [4]:
%run gen_files_array.ipynb

In [5]:
%run functions.ipynb

In [6]:
def get_wavelets(file, keys, wavelet = 'sym2', mlev = 2):
    wav = wt.Wavelet(wavelet)
    
    xstar, mu, stds = GP_prep(file, keys)
    
    for filt in keys: 
        coeffs = [np.array(wt.swt(mu[filt], wav, level=mlev)).flatten()]

    return np.concatenate(coeffs)

In [10]:
file = files[0]
filters = ['desg' , 'desi' , 'desr' , 'desz']

In [9]:
%%mprun 

samples_wavelets = []
results = []
keys = ['desg' , 'desi' , 'desr' , 'desz']


#results = []
for f in files[0:5]:
    
    results.append(get_wavelets(file,filters))

UsageError: Cell magic `%%mprun` not found.


In [12]:
end-start

59.534939765930176

In [13]:
%%time 

UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


In [ ]:
np.random.seed(9)
resp = []
clean_method = ''
wavelet = 'sym2'
mlev = 2
wav = wt.Wavelet(wavelet)
filters = ['desg' , 'desi' , 'desr' , 'desz']

i = 0
start = time.time()

with pm.Model() as model:
    eta = pm.HalfCauchy("eta", beta=2, testval=2.0)
    lengthscale = pm.Gamma("lenght", alpha=4, beta=0.1)
    cov = 500*eta**2 * pm.gp.cov.Matern32(1, lengthscale)        

    gp = pm.gp.Marginal(cov_func=cov)

    #mp = pm.find_MAP(include_transformed=True)
    
    for file in files[0:50]:
        data_dict, t = get_df(file)
        t = t.reshape(len(t),1)
            
        mus = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
        stds = {filters[0] : [], filters[1] : [], filters[2] : [], filters[3] : []}
    
        for band, dat in data_dict.items():
            #with error treatment std_dev
            x, y, yerr = get_xys(cleaning_df(data_dict[band], method = clean_method, clean_neg = True))
            x = x.reshape(len(x),1)
            
            y_ = gp.marginal_likelihood(str(band)+str(i), X=x, y=y, noise=yerr)
            mu, var = gp.predict(t,  diag=True)
            fit = pd.DataFrame({"t": t.flatten(),
                            "mu_total": mu,
                            "sd_total": np.sqrt(var)})
            #stds[band] = fit['mu_total']
            mus[band] = fit['sd_total']
            
        for filt in filters: 
            coeffs = [np.array(wt.swt(mus[filt], wav, level=mlev)).flatten()]            
        resp.append(np.concatenate(coeffs)) 
        i = i + 1
end = time.time()
print("Time running: " , (end - start)) 

In [13]:
def GP_matern32(t,x,y,yerr):
    np.random.seed(9)
    with pm.Model() as model:
        eta = pm.HalfCauchy("eta", beta=2, testval=2.0)
        lengthscale = pm.Gamma("lenght", alpha=4, beta=0.1)
        cov = 500*eta**2 * pm.gp.cov.Matern32(1, lengthscale)        

        gp = pm.gp.Marginal(cov_func=cov)

        y_ = gp.marginal_likelihood("y", X=x, y=y, noise=yerr)

        #mp = pm.find_MAP(include_transformed=True)

        mu, var = gp.predict(t,  diag=True)
        fit = pd.DataFrame({"t": t.flatten(),
                        "mu_total": mu,
                        "sd_total": np.sqrt(var)})
        return fit

In [33]:
(38.44158220291138*len(files)/10)/3600

22.761687951034972

In [34]:
start = time.time()
resp = get_wavelets(file,filters)
end = time.time()
print("Time running: " , (end - start)) 

Time running:  4.361891984939575


In [175]:
.013378877507315742 * len(files)/10

28.518415294594234

In [176]:
len(files)/8

2664.5

In [178]:
#tempo do 0 ao 100
0.1784640689028634 * 60

10.707844134171804

In [179]:
len(results)

100